In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
nltk.download('stopwords')
from sklearn import *

[nltk_data] Downloading package stopwords to C:\Users\Priyanshi
[nltk_data]     Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df=pd.read_csv("training-models/amazon.csv")

In [3]:
df.head()

,id,label,tweet
0,1,1,"Good case, Excellent value."
1,2,1,Great for the jawbone.
2,3,0,Tied to charger for conversations lasting more...
3,4,1,The mic is great.
4,5,0,I have to jiggle the plug to get it to line up...


In [4]:
df['label'].value_counts()

1    500
0    499
Name: label, dtype: int64

In [5]:
def clear_tweets(text):
    text=re.sub('<[^>]*>','',text)
    emojis=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text=re.sub('[\W]+',' ',text.lower()) +' '.join(emojis).replace('-','')

    return text   

In [8]:
df['cleared_tweets']=df['tweet'].apply(clear_tweets)
df.head()

,id,label,tweet,cleared_tweets
0,1,1,"Good case, Excellent value.",good case excellent value
1,2,1,Great for the jawbone.,great for the jawbone
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...
3,4,1,The mic is great.,the mic is great
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...


In [9]:
def tokenize_tweets(tweet):
    return tweet.split()

In [10]:
df['tokenized_tweets']=df['cleared_tweets'].apply(tokenize_tweets)
df.head()

,id,label,tweet,cleared_tweets,tokenized_tweets
0,1,1,"Good case, Excellent value.",good case excellent value,"[good, case, excellent, value]"
1,2,1,Great for the jawbone.,great for the jawbone,"[great, for, the, jawbone]"
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,"[tied, to, charger, for, conversations, lastin..."
3,4,1,The mic is great.,the mic is great,"[the, mic, is, great]"
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...,"[i, have, to, jiggle, the, plug, to, get, it, ..."


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Priyanshi
[nltk_data]     Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [14]:
def stopword_removal(token_tweet):
    filtered_list=[]
    for word in token_tweet:
            if word.casefold() not in stop_words:
                filtered_list.append(word)
    return filtered_list

In [19]:
df['stopwords_removed']=df['tokenized_tweets'].apply(stopword_removal)
df.head()

,id,label,tweet,cleared_tweets,tokenized_tweets,stopwords_removed
0,1,1,"Good case, Excellent value.",good case excellent value,"[good, case, excellent, value]","[good, case, excellent, value]"
1,2,1,Great for the jawbone.,great for the jawbone,"[great, for, the, jawbone]","[great, jawbone]"
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,"[tied, to, charger, for, conversations, lastin...","[tied, charger, conversations, lasting, 45, mi..."
3,4,1,The mic is great.,the mic is great,"[the, mic, is, great]","[mic, great]"
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...,"[i, have, to, jiggle, the, plug, to, get, it, ...","[jiggle, plug, get, line, right, get, decent, ..."


In [20]:
stemmer = PorterStemmer()

In [26]:
def stemming_tweets(tweet):
    stemmed_words = [stemmer.stem(word) for word in tweet]
    joined_str=""
    for word in stemmed_words:
        joined_str=joined_str+" "+word
    return joined_str

In [29]:
df['stemmed_tweets']=df['stopwords_removed'].apply(stemming_tweets)
df.head()

,id,label,tweet,cleared_tweets,tokenized_tweets,stopwords_removed,stemmed_tweets
0,1,1,"Good case, Excellent value.",good case excellent value,"[good, case, excellent, value]","[good, case, excellent, value]",good case excel valu
1,2,1,Great for the jawbone.,great for the jawbone,"[great, for, the, jawbone]","[great, jawbone]",great jawbon
2,3,0,Tied to charger for conversations lasting more...,tied to charger for conversations lasting more...,"[tied, to, charger, for, conversations, lastin...","[tied, charger, conversations, lasting, 45, mi...",tie charger convers last 45 minut major problem
3,4,1,The mic is great.,the mic is great,"[the, mic, is, great]","[mic, great]",mic great
4,5,0,I have to jiggle the plug to get it to line up...,i have to jiggle the plug to get it to line up...,"[i, have, to, jiggle, the, plug, to, get, it, ...","[jiggle, plug, get, line, right, get, decent, ...",jiggl plug get line right get decent volum


### Trying out new method

In [149]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)




In [150]:
# tweet_list = list(df['stemmed_tweets'].apply(tokenize_tweets))
# print(type(tweet_list))
# tokens_for_model = get_tweets_for_model(tweet_list)

pos_tweet_list=list(df['stemmed_tweets'][df['label']==1].apply(tokenize_tweets))
neg_tweet_list=list(df['stemmed_tweets'][df['label']==0].apply(tokenize_tweets))

pos_tokens = get_tweets_for_model(pos_tweet_list)
neg_tokens = get_tweets_for_model(neg_tweet_list)


In [151]:
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in pos_tokens]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in neg_tokens]

dataset = positive_dataset + negative_dataset


In [152]:
import random
random.shuffle(dataset)

In [153]:
train_data = dataset[:799]
test_data = dataset[800:]

In [154]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))



Accuracy is: 0.7688442211055276


In [160]:
custom = "Happy girls are prettiest and good"
custom=clear_tweets(custom)
custom=tokenize_tweets(custom)
custom=stopword_removal(custom)
custom=stemming_tweets(custom)
custom=tokenize_tweets(custom)
print(classifier.classify(dict([token, True] for token in custom)))

Positive


In [162]:
import pickle
data={"classifier":classifier}
with open ('saved_steps_new.pk1','wb') as file:
    pickle.dump(data,file)

In [163]:
with open('saved_steps_new.pk1','rb') as file:
    data=pickle.load(file)
model_loaded=data['classifier']
custom="This is so ugly. It is so bad remove it please"
custom=clear_tweets(custom)
custom=tokenize_tweets(custom)
custom=stopword_removal(custom)
custom=stemming_tweets(custom)
custom=tokenize_tweets(custom)
print(model_loaded.classify(dict([token, True] for token in custom)))

Negative


### Method ends here

In [93]:
X=df.cleared_tweets
Y=df.label.values
vectorizer = TfidfVectorizer(max_features=20000)
X_vec=vectorizer.fit_transform(X)

In [98]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_vec,Y,random_state=1,test_size=0.25,shuffle=False)

In [99]:
from sklearn.naive_bayes import MultinomialNB
mn=MultinomialNB().fit(X_train,y_train)
y_pred=mn.predict(X_test)

In [100]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.788


In [97]:
tweet="Congratulations on your wedding. We loved it"
tweet=clear_tweets(tweet)
tweet=tokenize_tweets(tweet)
tweet=stopword_removal(tweet)
tweet=stemming_tweets(tweet)
tweet=[tweet]
temp=pd.DataFrame(tweet,columns=['tweet'])

tweet_vec=vectorizer.fit_transform(temp.tweet)
tweet_pred=mn.predict(tweet_vec)


ValueError: dimension mismatch